![title](q10.png)

In [101]:
import sys; sys.path.append("../../")
from graphs.directedgraph import DirectedGraph

import numpy as np
from scipy import linalg
from fractions import Fraction
# print matrix with fractions instead of decimals
np.set_printoptions(formatter={'all':lambda x: str(Fraction(x).limit_denominator())})

In [ ]:
# Answer for part (a)

In [47]:
# Damping factor alpha = 7/8
alpha = 7/8

In [20]:
# Build the graph of the Web in the question 
# (index starts from 0 instead of 1)
W = DirectedGraph(8)

W.insertEdge(0,1)
W.insertEdge(0,2)
W.insertEdge(0,3)
W.insertEdge(0,7)
W.insertEdge(1,0)
W.insertEdge(1,2)
W.insertEdge(1,4)
W.insertEdge(1,6)
W.insertEdge(2,1)
W.insertEdge(2,3)
W.insertEdge(2,6)
W.insertEdge(3,4)
W.insertEdge(3,5)
W.insertEdge(5,0)
W.insertEdge(5,2)
W.insertEdge(5,6)
W.insertEdge(7,0)
W.insertEdge(7,2)
W.insertEdge(7,4)

print(W.getNumEdges())

19


In [33]:
# Get number of outgoing links from page i
def num_outgoing(W, i):
    ans = 0
    for j in range(W.getNumVertices()):
        if W.edgeExists(i, j):
            ans += 1
    return ans

In [45]:
# Build G_0, each row is the adjacency matrix (all weights = 1)
# divided by the number of outgoing links from vertex with the row index
M = W.getNumVertices()
G0 = np.zeros((M, M))

for i in range(M):
    for j in range(M):
        if W.edgeExists(i, j):
            G0[i][j] = 1
    out_links = num_outgoing(W, i)
    if out_links > 0:
        G0[i] /= out_links

print("G0 =")
print(G0)

G0 =
[[0 1/4 1/4 1/4 0 0 0 1/4]
 [1/4 0 1/4 0 1/4 0 1/4 0]
 [0 1/3 0 1/3 0 0 1/3 0]
 [0 0 0 0 1/2 1/2 0 0]
 [0 0 0 0 0 0 0 0]
 [1/3 0 1/3 0 0 0 1/3 0]
 [0 0 0 0 0 0 0 0]
 [1/3 0 1/3 0 1/3 0 0 0]]


In [46]:
# Fix the dangling node by adding a 1/M chance of jumping to any
# other page
G1 = np.array(G0)

for i in range(M):
    if num_outgoing(W, i) == 0:
        G1[i] = np.full((1, M), 1/M)

print("G1 =")
print(G1)

G1 =
[[0 1/4 1/4 1/4 0 0 0 1/4]
 [1/4 0 1/4 0 1/4 0 1/4 0]
 [0 1/3 0 1/3 0 0 1/3 0]
 [0 0 0 0 1/2 1/2 0 0]
 [1/8 1/8 1/8 1/8 1/8 1/8 1/8 1/8]
 [1/3 0 1/3 0 0 0 1/3 0]
 [1/8 1/8 1/8 1/8 1/8 1/8 1/8 1/8]
 [1/3 0 1/3 0 1/3 0 0 0]]


In [52]:
# Add the teleporation for rows corresponding to non-dangling pages
# If no edge exists (== 0), then it is replaced with (1-alpha)/M
# If edge exists (!= 0), then it is replaced with 
# alpha/#(P_i) + (1-alpha)/M
G2 = np.array(G1)

for i in range(M):
    P_i = num_outgoing(W, i)
    if P_i != 0:
        for j in range(M):
            if G2[i][j] == 0:
                G2[i][j] = (1-alpha)/M
            else:
                G2[i][j] = alpha/P_i + (1-alpha)/M

print("G2 =")
print(G2)              

G2 =
[[1/64 15/64 15/64 15/64 1/64 1/64 1/64 15/64]
 [15/64 1/64 15/64 1/64 15/64 1/64 15/64 1/64]
 [1/64 59/192 1/64 59/192 1/64 1/64 59/192 1/64]
 [1/64 1/64 1/64 1/64 29/64 29/64 1/64 1/64]
 [1/8 1/8 1/8 1/8 1/8 1/8 1/8 1/8]
 [59/192 1/64 59/192 1/64 1/64 1/64 59/192 1/64]
 [1/8 1/8 1/8 1/8 1/8 1/8 1/8 1/8]
 [59/192 1/64 59/192 1/64 59/192 1/64 1/64 1/64]]


In [87]:
# Equivalent way to make G:
# G = alpha(G0 + (1/M).d.eT) + ((1-alpha)/M).e.eT,
# where e = M x 1 vector of all 1, d = M x 1 vector of 1 if row 
# corresponds to dangling webpage and 0 if not
d = np.array([1 if num_outgoing(W, i) == 0 else 0 for i in range(M)]).reshape((M, 1))
e = np.full((M, 1), 1)

G = alpha*(G0 + 1/M * (d @ e.T)) + (1-alpha)/M * (e @ e.T)
print("G = ")
print(G)

G = 
[[1/64 15/64 15/64 15/64 1/64 1/64 1/64 15/64]
 [15/64 1/64 15/64 1/64 15/64 1/64 15/64 1/64]
 [1/64 59/192 1/64 59/192 1/64 1/64 59/192 1/64]
 [1/64 1/64 1/64 1/64 29/64 29/64 1/64 1/64]
 [1/8 1/8 1/8 1/8 1/8 1/8 1/8 1/8]
 [59/192 1/64 59/192 1/64 1/64 1/64 59/192 1/64]
 [1/8 1/8 1/8 1/8 1/8 1/8 1/8 1/8]
 [59/192 1/64 59/192 1/64 59/192 1/64 1/64 1/64]]


In [88]:
# G should be equal to G2, but use allclose instead of array_equal to 
# account for float precision inaccuracy
np.allclose(G, G2)

True

In [89]:
# Answer for part (b)

The Google matrix G is irreducible because all entries are nonzero, therefore its equivalent directed graph has a directed edge between any 2 vertices, and so the graph is strongly connected.
It is also aperiodic, because the graph is complete, so for every vertex there exists a loop containing that vertex of every length >= 2 (choose how many vertices to be in the loop and it will always be connectable).
Finally, G is finite since all of its entries are finite.

G is a Markov chain that is finite, irreducible and aperiodic, and so satisfies the following properties:
1. For every initial probability distribution of states q0, the value of qt = q0.G^t converges as t approaches infinity to a unique distribution q and satisfies q = q.G. This q is the same for any q0 and so is independent of q0.
2. If N(P_i, T) be the number of times the system has been in state P_i during T many transitions of a Markov chain, then as T approaches infinity, N(P_i, T) / T approaches q_i, the ith coordinate of the vector q.

The first property is the exact property required for the PageRank. If we let q start as any random row vector of size 1 x M and norm = 1, then qt = q0.G^t will converge to the row vector corresponding to the ranks of each page.

In [ ]:
# Answer for part (c)

In [135]:
# Solve the linear equation qG = q (q is a row vector)
# this is equivalent to finding the left eigenvector of G corresponding
# to eigenvalue 1 (should be the largest eigenvalue)
# at the end, normalise it so all ranks sum to 1

evals, evecs = linalg.eig(G, left=True, right=False)
# sort the eigenvalues and eigenvectors, 1 should be the largest
sort_idxs = np.argsort(evals)[::-1]
evals = evals[sort_idxs]
evecs = evecs[:,sort_idxs]

# The eigenvector corresponding to eigenvalue 1 should have all 
# real entries, it is on the first column <=> first row of transpose
ranks = np.real(evecs.T[0])
print(ranks)

[-0.35107991 -0.33574089 -0.42787864 -0.33574089 -0.41599921 -0.28103086
 -0.41435282 -0.21094295]


In [139]:
# Currently, the vector has length 1, but its entries are not positive 
# and do not sum to 1.
# To fix that, we can multiply by -1 and divide by the sum to get the
# final ranks.
ranks /= np.sum(ranks)

array([0.12661721, 0.12108518, 0.15431472, 0.12108518, 0.1500304 ,
       0.10135397, 0.14943662, 0.07607672])

In [140]:
# Print the ranks for each page 
for i in range(M):
    print(f"Page {i} has rank {ranks[i]}")

Page 0 has rank 0.12661720682025254
Page 1 has rank 0.1210851797341451
Page 2 has rank 0.1543147208121827
Page 3 has rank 0.12108517973414476
Page 4 has rank 0.15003039789256112
Page 5 has rank 0.10135397163901654
Page 6 has rank 0.1494366238704388
Page 7 has rank 0.07607671949725853
